# Read gene file

In [ ]:
f = open('ce11.fa', 'r')
gene_1 = ''
f.readline()
for line in f.readlines():
    if line[0] == '>':
        break
    gene_1 = gene_1 + line[:-1].upper()
f.close()

f = open('cb4.fa', 'r')
gene_2 = ''
f.readline()
for line in f.readlines():
    if line[0] == '>':
        break
    gene_2 = gene_2 + line[:-1].upper()
f.close()

print(len(gene_1), len(gene_2))

# Generate random gene sequence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(10)

random_pattern = False
seq_size = 100000

d = ['A', 'C', 'G', 'T']
if random_pattern:
    gene_1, gene_2 = '', ''
    for i in range(seq_size):
        gene_1 += d[random.randint(0, 3)]
    i = 0
    while i < seq_size:
        die = random.random()
        l = random.randint(1,10)
        # delete gene_1 for length l
        if die < 0.06:
            i += l
        # insert gene_1 for length l
        elif die < 0.12:
            for j in range(l):
                gene_2 += d[random.randint(0, 3)]
            i += 1
        # change 1 gene
        elif die < 0.18:
            gene_2 += d[random.randint(0, 3)]
            i += 1
        else:
            gene_2 += gene_1[i]
            i += 1
    while len(gene_2) < seq_size:
        gene_2 += d[random.randint(0, 3)]
            
    print(len(gene_1), len(gene_2))
    # print(gene_1)
    # print(gene_2)

# Build seed-look-up on reference sequence

In [ ]:
k = 8
pattern_len = 12
# base = ['A', 'C', 'G', 'T']
# seeds = []
# for i in range(4**k):
#     seed = ''
#     for j in range(k):
#         seed += base[i%4]
#         i = i // 4
#     seeds.append(seed)
# print(seeds)

# seed_to_index = {seeds[i]:i for i in range(len(seeds))}

def gen_pattern(k, length):
    l1 = np.zeros(length).astype(int)
    l2 = random.sample(range(length), k)
    l1[l2] = 1
    return l1
    
pattern = '1110100110010101111' # 12 of 19
pattern = gen_pattern(k, pattern_len)
print('Pattern:', pattern)

def seed_to_index(cut, pattern):
    if len(cut) != len(pattern):
        print('length mismatch')
        return None
    if 'N' in cut:
        print('Missing Nucleobase')
        return None
    base = {'A':0, 'C':1, 'G':2, 'T':3}
    index = 0
    j = 0
    # only count position selected by pattern
    for i in range(len(pattern)):
        if pattern[i] == 1:
            index += (4**j)*base[cut[i]]
            j += 1
    return index

# print(seed_to_index('ACTGACTGCTAG', pattern))

def check_hit(seq1, seq2, pattern):
    for i in range(len(pattern)):
        if pattern[i] and not (seq1[i] == seq2[i]):
            return False
    return True

In [ ]:
# position_table[i] is a list, memory start positions of seeds[i] in reference sequence
position_table = [[] for i in range(4**k)]
for i in range(len(gene_1)-pattern_len+1):
    print(f'Processing: {i}/{len(gene_1)-pattern_len}', "\r" , end=' ')
    cut = gene_1[i:i+pattern_len]
    index = seed_to_index(cut, pattern)
    position_table[index].append(i)
print(position_table)

# D-Soft filtering

In [ ]:
# reference: gene_1, query: gene_2
bin_size = 500
chunk_size = 500
threshold = 20

bin_num = len(gene_1) // bin_size
chunk_num = len(gene_2) // chunk_size

final_hits = []

# iterate every chunks
hit_positions = []
# hits count & last hit position of each bins
hit_num_in_bins = [0 for i in range(bin_num)]
last_hit_pos = [None for i in range(bin_num)]

for i in range(chunk_num):
    chunk = gene_2[i*chunk_size:(i+1)*chunk_size]
    print('\r')
    print(f'Processing chunk: {i+1}/{chunk_num}', end=' ')
    for j in range(len(chunk)-pattern_len+1):
        cut = chunk[j:j+pattern_len]
        seed_index = seed_to_index(cut, pattern)
        if seed_index == None:
            continue
        hits = np.array(position_table[seed_index])
        # for h in hits:
        #    print(f'hit at {i*chunk_size+j, h} {gene_1[h:h+k] == cut}')
        for hit in hits:
            if (hit-j) > 0 and (hit < bin_size*bin_num):
                hit_num_in_bins[(hit-j)//bin_size] += 1
                last_hit_pos[(hit-j)//bin_size] = (i*chunk_size+j, hit) # (position in gene_2, position in gene_1)
    # print(hit_num_in_bins)
    # print(last_hit_pos)
    for i in range(bin_num):
        if hit_num_in_bins[i] >= threshold:
            final_hits.append(last_hit_pos[i])
    hit_num_in_bins = [0 for i in range(bin_num)]
    
print(final_hits)
print(len(final_hits))

In [ ]:
for i in final_hits:
    seq1 = gene_1[i[1]:i[1]+pattern_len]
    seq2 = gene_2[i[0]:i[0]+pattern_len]
    print((i[1], i[0]), seq1, seq2, check_hit(seq1, seq2, pattern))